In [1]:
import os
import numpy as np
import xarray as xr
from glob import glob
import geopandas as gpd
from utils.functions import meter2deg
from utils.functions import pixc_geophy_cor
from utils.pixc2raster import pixc2raster
from utils.swot_data_mask import swot_pixc_mask
from utils.swot_data_filter import iter_IQR, pixc_height_local_filtering



In [2]:
dir_pixc = 'data/gyaring-lake/swot-pixc'
dir_raster = 'data/gyaring-lake/raster'
path_lake_vec = 'data/gyaring-lake/hydrolake_gyaring.gpkg'
path_raster_geoid_cor_smoothed = 'data/gyaring-lake/raster_geoid_cor_smoothed.nc'


In [3]:
## read vector file of the lake.
lake_gdf = gpd.read_file(path_lake_vec)
lake_gdf


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
0,1385,Gyaring,China,Asia,SWBD,1,0,520.58,143.97,1.78,...,1,9.0,19.118,2827.2,4290,-1.0,9516.1,97.312801,34.810857,"MULTIPOLYGON (((97.35592 35.01686, 97.35711 35..."


In [4]:
lake_decrease_gdf = lake_gdf.copy()
lon_center = lake_decrease_gdf.bounds.mean(axis=1).values
utm_zone = np.floor(lon_center/6)+31
epsg_code = f'326{int(utm_zone[0])}'
lake_decrease_gdf = lake_decrease_gdf.to_crs(epsg=epsg_code)
lake_decrease_gdf['geometry'] = lake_decrease_gdf.geometry.buffer(-200)
lake_decrease_gdf = lake_decrease_gdf.to_crs(epsg=4326)


In [5]:
## Check original .nc raster file.
paths_pixc = [p for p in glob(dir_pixc + '/*.nc') if '_masked' not in p]
paths_pixc = sorted(paths_pixc)
print(len(paths_pixc))


6


In [ ]:
## mask
pixcs_masked = []
for i, path in enumerate(paths_pixc):
  print(f'input file: {i} {path}')
  # Define the output path
  path_masked = path.split('.')[0]+'_masked.nc'
  pixc_nc = xr.open_dataset(path, group='pixel_cloud')
  pixc_masked = swot_pixc_mask(pixc_nc=pixc_nc, 
                                vars_sel=['latitude', 'longitude', 'height', 
                                          'solid_earth_tide', 'pole_tide', 
                                          'load_tide_fes', 'iono_cor_gim_ka', 'geoid',
                                    ],
                                region_gdf=lake_decrease_gdf)
  pixcs_masked.append(pixc_masked)


input file: 0 data/gyaring-lake/swot-pixc/SWOT_L2_HR_PIXC_014_146_093L_20240422T155501_20240422T155512_PIC0_01.nc
input file: 1 data/gyaring-lake/swot-pixc/SWOT_L2_HR_PIXC_015_146_093L_20240513T124005_20240513T124016_PIC0_01.nc
input file: 2 data/gyaring-lake/swot-pixc/SWOT_L2_HR_PIXC_016_146_093L_20240603T092510_20240603T092522_PIC0_01.nc
input file: 3 data/gyaring-lake/swot-pixc/SWOT_L2_HR_PIXC_017_146_093L_20240624T061014_20240624T061025_PIC0_01.nc
input file: 4 data/gyaring-lake/swot-pixc/SWOT_L2_HR_PIXC_020_146_093L_20240825T202526_20240825T202538_PIC0_01.nc
input file: 5 data/gyaring-lake/swot-pixc/SWOT_L2_HR_PIXC_021_146_093L_20240915T171033_20240915T171044_PIC0_01.nc


: 

In [ ]:
pixcs_masked_filterd = []
for pixc_masked in pixcs_masked:
    print(1)
    pixc_height_filter_ds = pixc_masked[['geoid', 'height']]
    ## 1. geophysical correction for height, and convert to orthometric height
    pixc_height_cor = pixc_geophy_cor(pixc_nc=pixc_masked)
    pixc_height_cor = pixc_height_cor - pixc_masked.geoid.values
    pixc_height_filter_ds = pixc_height_filter_ds.assign({'pixc_height_cor': (("points",), pixc_height_cor)})
    pixc_height_filter_ds.pixc_height_cor.attrs['description'] = 'PIXC height data with geophysical correction and geoid correction'

    ## 2. height filtering 
    ## 2.1 height filtering in global region using IQR method
    pixc_height_cor_filter1, IQR = iter_IQR(pixc_height_cor, IQR_thre=0.3, iter_max=5)
    pixc_height_cor_filter1 = pixc_height_cor_filter1.filled(np.nan)
    pixc_height_filter_ds = pixc_height_filter_ds.assign({'pixc_height_cor_filter1': (("points",), pixc_height_cor_filter1)})
    pixc_height_filter_ds.pixc_height_cor_filter1.attrs['description'] = 'PIXC height data with global filtering using IQR method'
    ## 2.2 height filtering in local region
    pixc_height_cor_filter2 = pixc_height_local_filtering(pixc_height=pixc_height_cor_filter1, 
                                              pixc_lonlat=(pixc_nc.longitude, pixc_nc.latitude), 
                                              thre=0.3, 
                                              radius_m=500)
    pixc_height_filter_ds = pixc_height_filter_ds.assign({'pixc_height_cor_filter2': (("points",), pixc_height_cor_filter2)})
    pixc_height_filter_ds.pixc_height_cor_filter2.attrs['description'] = 'PIXC height with both global and local filtering'
    pixcs_masked_filterd.append(pixc_height_filter_ds)


1


In [ ]:
## calculate the corrected geoid 
pixcs_geoid_cor_ds = []
for pixc_masked_filtered in pixcs_masked_filterd:
    pixc_geoid_cor_ds = pixc_masked_filtered[['pixc_height_cor_filter2', 'geoid']]
    ## calculate corrected geoid
    pixc_geoid_height_centered = pixc_masked_filtered['pixc_height_cor_filter2'].values - np.nanmean(pixc_masked_filtered['pixc_height_cor_filter2'])  # Centralization of the geoid height
    pixc_geoid_mean = np.nanmean(pixc_masked_filtered['geoid'])  
    pixc_geoid_cor = pixc_geoid_mean + pixc_geoid_height_centered    ## Geoid correction
    ## save as DataArray
    pixc_geoid_cor_ds = pixc_geoid_cor_ds.assign({'geoid_cor': (("points",), pixc_geoid_cor)})
    pixcs_geoid_cor_ds.append(pixc_geoid_cor_ds)



In [ ]:
### temporal smoothing for the geoid correction
xmin, ymin, xmax, ymax = lake_gdf.geometry[0].buffer(0.01).bounds
raster_extent = (xmin, xmax, ymin, ymax)
print('raster extent:', raster_extent)
lat_center = pixcs_geoid_cor_ds[0]['geoid_cor'].latitude.mean().values
res_lon, res_lat = meter2deg(meter=500, lat=lat_center)
print('resolution (lon, lat):', res_lon, res_lat)
rasters_geoid_cor = []
for pixc_geoid_cor in pixcs_geoid_cor_ds:
    raster_geoid_cor = pixc2raster(pixc_var = pixc_geoid_cor['geoid_cor'], 
                          raster_extent=raster_extent,
                          pixc_lonlat=(pixc_geoid_cor.longitude.values, pixc_geoid_cor.latitude.values), 
                          resolution=(res_lon, res_lat))
    raster_geoid_cor.attrs = pixc_geoid_cor.attrs
    rasters_geoid_cor.append(raster_geoid_cor)


In [ ]:
rasters_geoid_cor_da = xr.concat(rasters_geoid_cor, dim='date') 
rasters_geoid_cor_smoothed = rasters_geoid_cor_da.median(dim='date', keep_attrs=True)  # temporal smoothing
rasters_geoid_cor_smoothed.name = "geoid_cor_smoothed"
rasters_geoid_cor_smoothed
# ## Save as NetCDF file
rasters_geoid_cor_smoothed.to_netcdf(path_raster_geoid_cor_smoothed)
